# **Data Pre-processing**

In [3]:
import pandas as pd
data = pd.read_csv('/content/jewel.csv', header=0)

data = data.drop(['datetime','product_id','quantity','category_id','category_alias','Brand_id','price','user_id','gender'],axis=1) # Remove the Sample code column. axis{0 or ‘index’, 1 or ‘columns’}
print('Number of instances = %d' % (data.shape[0]))
print('Number of attributes = %d' % (data.shape[1]))
data.head(3)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Number of instances = 95911
Number of attributes = 4


,order_id,pc1,pc2,pc3
0,1.924720e+18,red,gold,diamond
1,1.924900e+18,yellow,gold,NaN
2,1.925510e+18,white,gold,sapphire


In [ ]:
print('Number of missing values:')
for col in data.columns:
    print('\t%s: %d' % (col,data[col].isna().sum()))

Number of missing values:
	order_id: 0
	pc1: 7660
	pc2: 5462
	pc3: 34058


In [4]:
sample = data.sample(frac=0.01,replace=False, random_state=1)
sample

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,order_id,pc1,pc2,pc3
80172,2.671960e+18,white,gold,diamond
76624,2.658140e+18,red,gold,NaN
67480,2.629430e+18,white,gold,diamond
24776,2.386990e+18,red,gold,topaz
77677,2.661820e+18,NaN,NaN,NaN
...,...,...,...,...
85679,2.698350e+18,red,gold,NaN
85199,2.696560e+18,white,gold,diamond
43460,2.502380e+18,red,gold,diamond
76569,2.658060e+18,red,gold,fianit


# **Apriori algorithm**

In [ ]:
import pandas as pd
import numpy as np
!pip install apyori
import apyori
from apyori import apriori

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5956 sha256=e0bf898cb779761fe09b1a110b6678a10eb7421d38bd2fbff68e84a24209fb8a
  Stored in directory: /root/.cache/pip/wheels/c4/1a/79/20f55c470a50bb3702a8cb7c94d8ada15573538c7f4baebe2d
Successfully built apyori


In [ ]:

l=[]
for i in range(1,959):
    l.append([str(sample.values[i,j]) for j in range(0,4)])

association_rules = apriori(l, min_support=0.0045, min_confidence=0.2, min_lift=3, min_length=2)
association_results = list(association_rules)

for i in range(0, len(association_results)):
    print(association_results[i][0])

frozenset({'sapphire', 'white'})
frozenset({'gold', 'sapphire', 'white'})


In [ ]:
for item in association_results:
    # first index of the inner list
    # Contains base item and add item
    pair = item[0]
    items = [x for x in pair]
    print("Rule: " + items[0] + " -> " + items[1])
    # second index of the inner list
    print("Support: " + str(item[1]))
    # third index of the list located at 0th position
    # of the third index of the inner list
    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("-----------------------------------------------------")

Rule: sapphire -> white
Support: 0.007306889352818371
Confidence: 0.5384615384615384
Lift: 3.1075069508804445
-----------------------------------------------------
Rule: gold -> sapphire
Support: 0.007306889352818371
Confidence: 0.5384615384615384
Lift: 3.1263403263403258
-----------------------------------------------------


# **FP-Growth algorithm**

In [12]:
import pandas as pd


sample.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(959, 3)

In [13]:
# printing the columns and few rows using head
sample.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,pc1,pc2,pc3
80172,white,gold,diamond
76624,red,gold,NaN
67480,white,gold,diamond
24776,red,gold,topaz
77677,NaN,NaN,NaN


In [14]:
# importing module
import numpy as np
# Gather All Items of Each Transactions into Numpy Array
transaction = []
for i in range(0, sample.shape[0]):
    for j in range(0, sample.shape[1]):
        transaction.append(sample.values[i,j])
# converting to numpy array
transaction = np.array(transaction)
print(transaction)

['white' 'gold' 'diamond' ... 'red' 'gold' 'emerald']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
import pandas as pd
#  Transform Them a Pandas DataFrame
df = pd.DataFrame(transaction, columns=["items"])
# Put 1 to Each Item For Making Countable Table, to be able to perform Group By
df["incident_count"] = 1
#  Delete NaN Items from Dataset
indexNames = df[df['items'] == "nan" ].index
df.drop(indexNames , inplace=True)
# Making a New Appropriate Pandas DataFrame for Visualizations
df_table = df.groupby("items").sum().sort_values("incident_count", ascending=False).reset_index()
#  Initial Visualizations
df_table.head(5).style.background_gradient(cmap='Blues')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,items,incident_count
0,gold,904
1,red,702
2,diamond,292
3,white,167
4,fianit,130


In [43]:
%pip install mlxtend --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.3 MB/s eta 0:00:00
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [17]:
# Transform Every Transaction to Seperate List & Gather Them into Numpy Array
transaction = []
for i in range(sample.shape[0]):
    transaction.append([str(sample.values[i,j]) for j in range(sample.shape[1])])
# creating the numpy array of the transactions
transaction = np.array(transaction)
# importing the required module
from mlxtend.preprocessing import TransactionEncoder
# initializing the transactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transaction).transform(transaction)
sample = pd.DataFrame(te_ary, columns=te.columns_)
# dataset after encoded
sample.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,amber,amethyst,chrysolite,diamond,emerald,fianit,garnet,gold,mix,nan,...,quartz,red,ruby,sapphire,silver,sitall,spinel,topaz,white,yellow
0,False,False,False,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,True,False,True,...,False,True,False,False,False,False,False,False,False,False
2,False,False,False,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,True,False,False
4,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [18]:
#Importing Libraries

from mlxtend.frequent_patterns import fpgrowth
#running the fpgrowth algorithm
res=fpgrowth(sample,min_support=0.045, use_colnames=True)
# printing top 10
res.head(10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.942649,(gold)
1,0.304484,(diamond)
2,0.174140,(white)
3,0.732013,(red)
4,0.361835,(nan)
5,0.059437,(topaz)
6,0.135558,(fianit)
7,0.304484,"(diamond, gold)"
8,0.194995,"(diamond, red)"
9,0.194995,"(diamond, red, gold)"


In [19]:
# importing required module
from mlxtend.frequent_patterns import association_rules
# creating asssociation rules
res=association_rules(res, metric="lift", min_threshold=1)
# printing association rules
res

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(diamond),(gold),0.304484,0.942649,0.304484,1.000000,1.060841,0.017463,inf,0.082459
1,(gold),(diamond),0.942649,0.304484,0.304484,0.323009,1.060841,0.017463,1.027364,1.000000
2,"(diamond, red)",(gold),0.194995,0.942649,0.194995,1.000000,1.060841,0.011183,inf,0.071244
3,(gold),"(diamond, red)",0.942649,0.194995,0.194995,0.206858,1.060841,0.011183,1.014958,1.000000
4,(white),(gold),0.174140,0.942649,0.173097,0.994012,1.054488,0.008944,9.577685,0.062568
5,(gold),(white),0.942649,0.174140,0.173097,0.183628,1.054488,0.008944,1.011623,0.900986
6,(white),(diamond),0.174140,0.304484,0.096976,0.556886,1.828952,0.043953,1.569611,0.548808
7,(diamond),(white),0.304484,0.174140,0.096976,0.318493,1.828952,0.043953,1.211815,0.651658
8,"(white, diamond)",(gold),0.096976,0.942649,0.096976,1.000000,1.060841,0.005562,inf,0.063510
9,"(white, gold)",(diamond),0.173097,0.304484,0.096976,0.560241,1.839969,0.044271,1.581585,0.552075


In [20]:
# Sort values based on confidence
res.sort_values("confidence",ascending=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(diamond),(gold),0.304484,0.942649,0.304484,1.000000,1.060841,0.017463,inf,0.082459
8,"(white, diamond)",(gold),0.096976,0.942649,0.096976,1.000000,1.060841,0.005562,inf,0.063510
36,"(red, fianit)",(gold),0.116788,0.942649,0.116788,1.000000,1.060841,0.006698,inf,0.064935
24,"(topaz, red)",(gold),0.047967,0.942649,0.047967,1.000000,1.060841,0.002751,inf,0.060241
16,"(red, nan)",(gold),0.247132,0.942649,0.247132,1.000000,1.060841,0.014173,inf,0.076177
14,(red),(gold),0.732013,0.942649,0.732013,1.000000,1.060841,0.041982,inf,0.214008
20,(topaz),(gold),0.059437,0.942649,0.059437,1.000000,1.060841,0.003409,inf,0.060976
2,"(diamond, red)",(gold),0.194995,0.942649,0.194995,1.000000,1.060841,0.011183,inf,0.071244
4,(white),(gold),0.174140,0.942649,0.173097,0.994012,1.054488,0.008944,9.577685,0.062568
31,(fianit),(gold),0.135558,0.942649,0.134515,0.992308,1.052680,0.006732,7.455683,0.057892
